Project overview 
Goal = Recommend Books
Needs: 
List of books we like
Data on Book Ratings

Data used = GOODREADS Book data

Files Used :
Good Reads  Interaction .csv
Good Reads Books .json
Book Id Map .csv


STEPs for Project:
1: Search For Book
2:Create Book Lists
3:Recommend Books



# Exploring Data in Python

In [21]:
!wc -l goodreads_books.json.gz

7588375 goodreads_books.json.gz


In [22]:
!ls -lh | grep goodreads_books.json.gz

-rw-rw-r-- 1 threat_hunter threat_hunter 2.0G Aug 19 00:13 goodreads_books.json.gz


In [23]:
import gzip

In [24]:
with gzip.open('goodreads_books.json.gz','r') as f:
    line = f.readline()

In [25]:
line

b'{"isbn": "0312853122", "text_reviews_count": "1", "series": [], "country_code": "US", "language_code": "", "popular_shelves": [{"count": "3", "name": "to-read"}, {"count": "1", "name": "p"}, {"count": "1", "name": "collection"}, {"count": "1", "name": "w-c-fields"}, {"count": "1", "name": "biography"}], "asin": "", "is_ebook": "false", "average_rating": "4.00", "kindle_asin": "", "similar_books": [], "description": "", "format": "Paperback", "link": "https://www.goodreads.com/book/show/5333265-w-c-fields", "authors": [{"author_id": "604031", "role": ""}], "publisher": "St. Martin\'s Press", "num_pages": "256", "publication_day": "1", "isbn13": "9780312853129", "publication_month": "9", "edition_information": "", "publication_year": "1984", "url": "https://www.goodreads.com/book/show/5333265-w-c-fields", "image_url": "https://images.gr-assets.com/books/1310220028m/5333265.jpg", "book_id": "5333265", "ratings_count": "3", "work_id": "5400751", "title": "W.C. Fields: A Life on Film", "t

In [26]:
import json
json.loads(line)

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
  {'count': '1', 'name': 'p'},
  {'count': '1', 'name': 'collection'},
  {'count': '1', 'name': 'w-c-fields'},
  {'count': '1', 'name': 'biography'}],
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'edition_information': '',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'work_id': '5400751',
 'title': '

# Pasring our Book METADATA

In [27]:
def parse_fields(line):
    data = json.loads(line)
    return{
        "book_id":data["book_id"],
        "title":data["title_without_series"],
        "ratings":data["ratings_count"],
        "url":data["url"],
        "cover_image":data["image_url"]
        
    }

In [28]:
books_titles =[]
with gzip.open("goodreads_books.json.gz",'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        fields = parse_fields(line)
        
        try:
            ratings = int(fields["ratings"])
        except ValueError:
            continue
        if ratings > 15:
            books_titles.append(fields)
    

# Processing Book METADATA with Pandas

In [29]:
import pandas as pd

titles = pd.DataFrame.from_dict(books_titles)

In [30]:
titles['ratings'] = pd.to_numeric(titles['ratings'])

In [31]:
titles["mod_title"] = titles["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)

In [32]:
titles["mod_title"] = titles["mod_title"].str.lower()

In [33]:
titles["mod_title"] = titles["mod_title"].str.replace("\s+", " ", regex=True)

In [34]:
titles = titles[titles["mod_title"].str.len() > 0]


In [35]:
titles.to_json("books_titles.json")

In [36]:
titles

,book_id,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook
...,...,...,...,...,...,...
1308952,17805813,"Ondine (Ondine Quartet, #0.5)",327,https://www.goodreads.com/book/show/17805813-o...,https://images.gr-assets.com/books/1379766592m...,ondine ondine quartet 05
1308953,331839,Jacqueline Kennedy Onassis: Friend of the Arts,18,https://www.goodreads.com/book/show/331839.Jac...,https://s.gr-assets.com/assets/nophoto/book/11...,jacqueline kennedy onassis friend of the arts
1308954,2685097,The Spaniard's Blackmailed Bride,112,https://www.goodreads.com/book/show/2685097-th...,https://s.gr-assets.com/assets/nophoto/book/11...,the spaniards blackmailed bride
1308955,2342551,The Children's Classic Poetry Collection,36,https://www.goodreads.com/book/show/2342551.Th...,https://s.gr-assets.com/assets/nophoto/book/11...,the childrens classic poetry collection


# Building A Book Search Engine

Term Frequency  * Inverse Document Frequency  -- Matrix

Example:
The                   The      Harry    Potter
1    0      0         log(3/3) log(3/2) log(3/1)
                                        0.23856062736 
The Harry Potter *     
1    1     1
The Harry
1    1     0


Harry = 2  *  0.23856062736  =      0.47712125472


In [38]:
pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached scikit_learn-1.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [39]:

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

tfidf = vectorizer.fit_transform(titles["mod_title"])

In [64]:
tfidf

<1302659x233827 sparse matrix of type '<class 'numpy.float64'>'
	with 6207224 stored elements in Compressed Sparse Row format>

In [49]:
'''
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re


query ="fire upon the deep"

processed = re.sub("[^a-zA-Z0-9 ]","",query.lower())
query_vec = vectorizer.transform([processed])
similarity = cosine_similarity(query_vec,tfidf).flatten()

indices = np.argpartition(similarity, -10)[-10:]
results = titles.iloc[indices]
results = results.sort_values("ratings", ascending=False)
'''



'\nfrom sklearn.metrics.pairwise import cosine_similarity\nimport numpy as np\nimport re\n\n\nquery ="fire upon the deep"\n\nprocessed = re.sub("[^a-zA-Z0-9 ]","",query.lower())\nquery_vec = vectorizer.transform([processed])\nsimilarity = cosine_similarity(query_vec,tfidf).flatten()\n\nindices = np.argpartition(similarity, -10)[-10:]\nresults = titles.iloc[indices]\nresults = results.sort_values("ratings", ascending=False)\n'

In [50]:
#similarity
#results

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)


#query ="fire upon the deep"
def search(query,vectorizer):
    processed = re.sub("[^a-zA-Z0-9 ]","",query.lower())
    query_vec = vectorizer.transform([processed])
    similarity = cosine_similarity(query_vec,tfidf).flatten()

    indices = np.argpartition(similarity, -10)[-10:]
    results = titles.iloc[indices]
    results = results.sort_values("ratings", ascending=False)
    #return results.head(5)
    return results.head(5).style.format({'url': make_clickable, 'cover_image': show_image})


In [63]:
search('Harry Potter',vectorizer)

,book_id,title,ratings,url,cover_image,mod_title
129532,7809996,"هاري بوتر وحجرة الأسرار (Harry Potter, #2)",1117,Goodreads,,harry potter 2
1044938,49869,"هاري بوتر وسجين أزكابان (Harry Potter, #3)",1023,Goodreads,,harry potter 3
19737,49839,"هاري بوتر وكأس النار (Harry Potter, #4)",957,Goodreads,,harry potter 4
903847,70355,"هاري بوتر وجماعة العنقاء (Harry Potter, #5)",955,Goodreads,,harry potter 5
138743,14560521,"הארי פוטר והאסיר מאזקבאן (Harry Potter, #3)",36,Goodreads,,harry potter 3


In [ ]:
liked_books = ["4408", "31147619", "29983711", "9401317", "9317691", "8153988", "20494944"]